# Połączenia
<img src = "https://i.imgur.com/3j8BPdc.png" style = "height:300px">

# Import

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Funkcje

In [2]:
def caluclate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def reset_by_hands(left_angle, right_angle, l_wrist_y, r_wrist_y, nose_y):
    return left_angle > 170 and right_angle > 170 and l_wrist_y < nose_y and r_wrist_y < nose_y

In [104]:
cap = cv2.VideoCapture(0)
width, height = 800, 600
cap.set(3, width)
cap.set(4, height)
counter_l, counter_r = 0, 0
stage_l, stage_r = None, None


with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    while cap.isOpened():
        _, frame = cap.read()
        
        # Recolor to rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Detection
        results = pose.process(image)
        
        # Recolor to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            left_angle = caluclate_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = caluclate_angle(right_shoulder, right_elbow, right_wrist)
            
#             cv2.putText(image, "{:.2f}".format(left_angle), tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
#                                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
#             cv2.putText(image, "{:.2f}".format(right_angle), tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
#                                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
            # Curl counter logic
            # Left
            if left_angle > 160:
                stage_l = "down"
            if left_angle < 30 and stage_l == "down":
                stage_l = "up"
                counter_l += 1
            # Right    
            if right_angle > 160:
                stage_r = "down"
            if right_angle < 30 and stage_r == "down":
                stage_r = "up"
                counter_r += 1
            
            text_color = (255, 255, 255)
            color = (0, 0, 0)
            cv2.rectangle(image, (0, 0), (225, 73), color, -1)
            cv2.rectangle(image, (width-225, 0), (width, 73), color, -1)
            cv2.putText(image, "Lewa", (5,22), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            cv2.putText(image, "Prawa", (width - 220,22), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            cv2.putText(image, str(counter_l), (45,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, str(counter_r), (width - 180,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, "{:.2f}".format(left_angle), (95,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, "{:.2f}".format(right_angle), (width - 130,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            
            l_wrist_y = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            r_wrist_y = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            nose_y = landmarks[mp_pose.PoseLandmark.NOSE.value].y
            
            if reset_by_hands(left_angle, right_angle, l_wrist_y, r_wrist_y) or cv2.waitKey(1) & 0xFF == ord('r'):
                counter_l, counter_r = 0, 0
            
        except:
            pass
        
        # Render
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow("Kamera", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
results.pose_landmarks

landmark {
  x: 0.23921050131320953
  y: 0.6571065783500671
  z: -1.5740021467208862
  visibility: 0.9996506571769714
}
landmark {
  x: 0.28697121143341064
  y: 0.5359030961990356
  z: -1.620990514755249
  visibility: 0.9994696378707886
}
landmark {
  x: 0.31641095876693726
  y: 0.5258700251579285
  z: -1.6207456588745117
  visibility: 0.999497652053833
}
landmark {
  x: 0.34560585021972656
  y: 0.5182974338531494
  z: -1.620969295501709
  visibility: 0.9995776414871216
}
landmark {
  x: 0.18371503055095673
  y: 0.5622467398643494
  z: -1.545202612876892
  visibility: 0.999261200428009
}
landmark {
  x: 0.15635468065738678
  y: 0.5643437504768372
  z: -1.5441467761993408
  visibility: 0.9990081787109375
}
landmark {
  x: 0.13675887882709503
  y: 0.5643138289451599
  z: -1.5440293550491333
  visibility: 0.9989499449729919
}
landmark {
  x: 0.4299249053001404
  y: 0.5145232081413269
  z: -1.407876968383789
  visibility: 0.9997311234474182
}
landmark {
  x: 0.1258457601070404
  y: 0.55983

In [67]:
mp_pose.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [7]:
left_shoulder

[0.6920495629310608, 0.840080201625824]

In [59]:
mp_drawing.draw_landmarks??

# Wersja 2

In [107]:
cap = cv2.VideoCapture(0)

# Const
width, height = 800, 600
cap.set(3, width)
cap.set(4, height)
counter_l, counter_r, counter_p = 0, 0, 0
stage_l, stage_r, stage_p = None, None, None


with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    while cap.isOpened():
        _, frame = cap.read()
        
        # Recolor to rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Detection
        results = pose.process(image)
        
        # Recolor to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            
            # Left arm cords
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Right arm cords
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Arm angles
            left_angle = caluclate_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = caluclate_angle(right_shoulder, right_elbow, right_wrist)
            
            # Left leg cords
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            squat_angle_l = caluclate_angle(left_hip, left_knee, left_ankle)
            
            # Right leg cords
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Squat angle
            squat_angle_r = caluclate_angle(right_hip, right_knee, right_ankle)
                
            # Curl counter logic
            # Left
            if left_angle > 160:
                stage_l = "down"
            if left_angle < 30 and stage_l == "down":
                stage_l = "up"
                counter_l += 1

            # Right    
            if right_angle > 160:
                stage_r = "down"
            if right_angle < 30 and stage_r == "down":
                stage_r = "up"
                counter_r += 1
            
            # Squat counter logic
            if squat_angle_l and squat_angle_r > 90:
                stage_p = "up"
            if squat_angle_l < 90 and squat_angle_r < 90 and stage_p == "up":
                stage_p = "down"
                counter_p +=1
            
            text_color = (255, 255, 255)
            color = (0, 0, 0)
            
            # Rect
            cv2.rectangle(image, (0, 0), (225, 73), color, -1)
            cv2.rectangle(image, (width-225, 0), (width, 73), color, -1)
            cv2.rectangle(image, (width-225, height-73), (width, height), color, -1)
            
            # Name
            cv2.putText(image, "Lewa", (5,22), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            cv2.putText(image, "Prawa", (width - 220,22), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            cv2.putText(image, "Przysiad", (width-220, height-42), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            
            # Counters
            cv2.putText(image, str(counter_l), (45,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, str(counter_r), (width - 180,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, str(counter_p), (width - 180, height-10), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            
            # Angles
            cv2.putText(image, "{:.2f}".format(left_angle), (95,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, "{:.2f}".format(right_angle), (width - 130,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, "{:.2f}".format(squat_angle_l), (width - 130,height-10), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            
            # Y values of wrists and nose
            l_wrist_y = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            r_wrist_y = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            nose_y = landmarks[mp_pose.PoseLandmark.NOSE.value].y
            
            # Reset function
            if reset_by_hands(left_angle, right_angle, l_wrist_y, r_wrist_y, nose_y) or cv2.waitKey(1) & 0xFF == ord('r'):
                counter_l, counter_r, counter_p = 0, 0, 0
        
        except:
            pass
        
        # Render
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Left arm
        ## Green when bent
        if left_angle < 30:
            cv2.line(image, (int(left_shoulder[0]*width), int(left_shoulder[1]*height)), (int(left_elbow[0]*width), int(left_elbow[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(left_elbow[0]*width), int(left_elbow[1]*height)), (int(left_wrist[0]*width), int(left_wrist[1]*height)), (0, 255, 0), 5)
       
        ## Red when not bent
        if left_angle > 30:
            cv2.line(image, (int(left_shoulder[0]*width), int(left_shoulder[1]*height)), (int(left_elbow[0]*width), int(left_elbow[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(left_elbow[0]*width), int(left_elbow[1]*height)), (int(left_wrist[0]*width), int(left_wrist[1]*height)), (0, 0, 255), 5)
      
        # Right arm 
        ## Green when bent
        if right_angle < 30:
            cv2.line(image, (int(right_shoulder[0]*width), int(right_shoulder[1]*height)), (int(right_elbow[0]*width), int(right_elbow[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(right_elbow[0]*width), int(right_elbow[1]*height)), (int(right_wrist[0]*width), int(right_wrist[1]*height)), (0, 255, 0), 5)
       
        ## Red when not bent
        if right_angle > 30:
            cv2.line(image, (int(right_shoulder[0]*width), int(right_shoulder[1]*height)), (int(right_elbow[0]*width), int(right_elbow[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(right_elbow[0]*width), int(right_elbow[1]*height)), (int(right_wrist[0]*width), int(right_wrist[1]*height)), (0, 0, 255), 5)
                    
        # Left leg 
        ## Green when bent
        if squat_angle_l < 90:
            cv2.line(image, (int(left_hip[0]*width), int(left_hip[1]*height)), (int(left_knee[0]*width), int(left_knee[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(left_knee[0]*width), int(left_knee[1]*height)), (int(left_ankle[0]*width), int(left_ankle[1]*height)), (0, 255, 0), 5)
       
        ## Red when not bent
        if squat_angle_l > 90:
            cv2.line(image, (int(left_hip[0]*width), int(left_hip[1]*height)), (int(left_knee[0]*width), int(left_knee[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(left_knee[0]*width), int(left_knee[1]*height)), (int(left_ankle[0]*width), int(left_ankle[1]*height)), (0, 0, 255), 5)
        
        # Right leg 
        ## Green when bent
        if squat_angle_r < 90:
            cv2.line(image, (int(right_hip[0]*width), int(right_hip[1]*height)), (int(right_knee[0]*width), int(right_knee[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(right_knee[0]*width), int(right_knee[1]*height)), (int(right_ankle[0]*width), int(right_ankle[1]*height)), (0, 255, 0), 5)
            
        ## Red when not bent
        if squat_angle_r > 90:
            cv2.line(image, (int(right_hip[0]*width), int(right_hip[1]*height)), (int(right_knee[0]*width), int(right_knee[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(right_knee[0]*width), int(right_knee[1]*height)), (int(right_ankle[0]*width), int(right_ankle[1]*height)), (0, 0, 255), 5)
             

        cv2.imshow("Kamera", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Wersja 3

In [8]:
cap = cv2.VideoCapture(0)

# Const
width, height = 800, 600
cap.set(3, width)
cap.set(4, height)
counter_l, counter_r, counter_p = 0, 0, 0
stage_l, stage_r, stage_p = None, None, None


with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    while cap.isOpened():
        _, frame = cap.read()
        
        # Recolor to rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Detection
        results = pose.process(image)
        
        # Recolor to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            
            # Left arm cords
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Right arm cords
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Arm angles
            left_angle = caluclate_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = caluclate_angle(right_shoulder, right_elbow, right_wrist)
            
            # Left leg cords
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            squat_angle_l = caluclate_angle(left_hip, left_knee, left_ankle)
            
            # Right leg cords
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Squat angle
            squat_angle_r = caluclate_angle(right_hip, right_knee, right_ankle)
                
            # Curl counter logic
            # Left
            if left_angle > 160:
                stage_l = "down"
            if left_angle < 30 and stage_l == "down":
                stage_l = "up"
                counter_l += 1

            # Right    
            if right_angle > 160:
                stage_r = "down"
            if right_angle < 30 and stage_r == "down":
                stage_r = "up"
                counter_r += 1
            
            # Squat counter logic
            if squat_angle_l and squat_angle_r > 90:
                stage_p = "up"
            if squat_angle_l < 90 and squat_angle_r < 90 and stage_p == "up":
                stage_p = "down"
                counter_p +=1
            
            text_color = (255, 255, 255)
            color = (0, 0, 0)
            
            # Rect
            cv2.rectangle(image, (0, 0), (225, 73), color, -1)
            cv2.rectangle(image, (width-225, 0), (width, 73), color, -1)
            cv2.rectangle(image, (width-225, height-73), (width, height), color, -1)
            
            # Name
            cv2.putText(image, "Lewa", (5,22), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            cv2.putText(image, "Prawa", (width - 220,22), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            cv2.putText(image, "Przysiad", (width-220, height-42), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 1, cv2.LINE_AA)
            
            # Counters
            cv2.putText(image, str(counter_l), (45,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, str(counter_r), (width - 180,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, str(counter_p), (width - 180, height-10), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            
            # Angles
            cv2.putText(image, "{:.2f}".format(left_angle), (95,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, "{:.2f}".format(right_angle), (width - 130,58), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            cv2.putText(image, "{:.2f}".format(squat_angle_l), (width - 130,height-10), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            
            # Y values of wrists and nose
            l_wrist_y = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            r_wrist_y = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            nose_y = landmarks[mp_pose.PoseLandmark.NOSE.value].y
            
            # Reset function
            if reset_by_hands(left_angle, right_angle, l_wrist_y, r_wrist_y, nose_y) or cv2.waitKey(1) & 0xFF == ord('r'):
                counter_l, counter_r, counter_p = 0, 0, 0
        
        except:
            pass
        
        # Render
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Left arm
        ## Green when bent
        if left_angle < 30:
            cv2.line(image, (int(left_shoulder[0]*width), int(left_shoulder[1]*height)), (int(left_elbow[0]*width), int(left_elbow[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(left_elbow[0]*width), int(left_elbow[1]*height)), (int(left_wrist[0]*width), int(left_wrist[1]*height)), (0, 255, 0), 5)
       
        ## Red when not bent
        if left_angle > 30:
            cv2.line(image, (int(left_shoulder[0]*width), int(left_shoulder[1]*height)), (int(left_elbow[0]*width), int(left_elbow[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(left_elbow[0]*width), int(left_elbow[1]*height)), (int(left_wrist[0]*width), int(left_wrist[1]*height)), (0, 0, 255), 5)
      
        # Right arm 
        ## Green when bent
        if right_angle < 30:
            cv2.line(image, (int(right_shoulder[0]*width), int(right_shoulder[1]*height)), (int(right_elbow[0]*width), int(right_elbow[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(right_elbow[0]*width), int(right_elbow[1]*height)), (int(right_wrist[0]*width), int(right_wrist[1]*height)), (0, 255, 0), 5)
       
        ## Red when not bent
        if right_angle > 30:
            cv2.line(image, (int(right_shoulder[0]*width), int(right_shoulder[1]*height)), (int(right_elbow[0]*width), int(right_elbow[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(right_elbow[0]*width), int(right_elbow[1]*height)), (int(right_wrist[0]*width), int(right_wrist[1]*height)), (0, 0, 255), 5)
                    
        # Left leg 
        ## Green when bent
        if squat_angle_l < 90:
            cv2.line(image, (int(left_hip[0]*width), int(left_hip[1]*height)), (int(left_knee[0]*width), int(left_knee[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(left_knee[0]*width), int(left_knee[1]*height)), (int(left_ankle[0]*width), int(left_ankle[1]*height)), (0, 255, 0), 5)
       
        ## Red when not bent
        if squat_angle_l > 90:
            cv2.line(image, (int(left_hip[0]*width), int(left_hip[1]*height)), (int(left_knee[0]*width), int(left_knee[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(left_knee[0]*width), int(left_knee[1]*height)), (int(left_ankle[0]*width), int(left_ankle[1]*height)), (0, 0, 255), 5)
        
        # Right leg 
        ## Green when bent
        if squat_angle_r < 90:
            cv2.line(image, (int(right_hip[0]*width), int(right_hip[1]*height)), (int(right_knee[0]*width), int(right_knee[1]*height)), (0, 255, 0), 5)
            cv2.line(image, (int(right_knee[0]*width), int(right_knee[1]*height)), (int(right_ankle[0]*width), int(right_ankle[1]*height)), (0, 255, 0), 5)
            
        ## Red when not bent
        if squat_angle_r > 90:
            cv2.line(image, (int(right_hip[0]*width), int(right_hip[1]*height)), (int(right_knee[0]*width), int(right_knee[1]*height)), (0, 0, 255), 5)
            cv2.line(image, (int(right_knee[0]*width), int(right_knee[1]*height)), (int(right_ankle[0]*width), int(right_ankle[1]*height)), (0, 0, 255), 5)
             

        cv2.imshow("Kamera", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()